# _wtie_: Introduction to the package wtie: 软件包介绍

In this tutorial we work on data from the Well _Torosa1_ of the open [Poseidon](https://terranubis.com/datainfo/NW-Shelf-Australia-Poseidon-3D) dataset. Data is saved in the folder **data/tutorial**.
在本教程中，我们使用来自开放的 Poseidon 数据集的 Torosa1 井的数据。数据保存在 data/tutorial 文件夹中。

### Load packages 加载包


In [ ]:
import os
import sys
from pathlib import Path
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from wtie import grid, tie, viz
from wtie.utils.datasets import tutorial

# uncomment if your browser supports it
# %matplotlib notebook

### Load data 加载数据


In [ ]:
# data path
folder = Path("../data/tutorial")

# inputs is a class that stores the useful input data to perform a well tie
inputs = tutorial.load_poseidon_data(folder)

In [ ]:
pprint(inputs.__dict__)

In [ ]:
inputs.plot_inputs(figsize=(7.5, 3.5));

### Load neural newtork to perfrom wavelet extraction 加载用于子波提取的神经网络


In [ ]:
# neural network's pre-trained weights
model_state_dict = folder / "trained_net_state_dict.pt"
assert model_state_dict.is_file()

# network training paramters
with open(folder / "network_parameters.yaml", "r") as yaml_file:
    training_parameters = yaml.load(yaml_file, Loader=yaml.Loader)

wavelet_extractor = tutorial.load_wavelet_extractor(training_parameters, model_state_dict)

### Filter the well logs 过滤测井曲线


In [ ]:
filtered_logset_md = tie.filter_md_logs(inputs.logset_md, median_size=21, threshold=2.0, std=2.0, std2=None)

viz.plot_logsets_overlay(inputs.logset_md, filtered_logset_md, figsize=(6, 5))

### Depth to time convertion 深度-时间转换


In [ ]:
logset_twt = tie.convert_logs_from_md_to_twt(
    filtered_logset_md, inputs.wellpath, inputs.table, dt=wavelet_extractor.expected_sampling
)
viz.plot_logset(logset_twt, figsize=(6, 5));

In [ ]:
# compute velocities from Vp and checkshot tables
fig, ax = viz.plot_trace(logset_twt.Vp, plot_params=dict(label="Sonic log"))
viz.plot_trace(inputs.table.slope_velocity_twt(), fig_axes=(fig, ax), plot_params=dict(label="Checkshot velocity"))
ax.legend(loc="best")
ax.set_ylim((logset_twt.Vp.basis[0], logset_twt.Vp.basis[-1]))

### Compute vertical incidence relfectivity 计算垂直入射反射系数


In [ ]:
r0 = tie.compute_reflectivity(logset_twt)
viz.plot_reflectivity(r0)  # type: ignore

### Interpolate seismic and find intersection with reflectivity 插值地震道并与反射系数对齐


In [ ]:
seismic_sinc = tie.resample_seismic(inputs.seismic, wavelet_extractor.expected_sampling)
seis_match, r0_match = tie.match_seismic_and_reflectivity(seismic_sinc, r0)  # type: ignore

In [ ]:
viz.plot_seismic_and_reflectivity(seis_match, r0_match, normalize=True, title="Real seismic and reflectivity")  # type: ignore

### Compute seismic wavelet 计算地震子波


In [ ]:
from wtie.optimize.wavelet import compute_expected_wavelet

In [ ]:
pred_wlt = compute_expected_wavelet(evaluator=wavelet_extractor, seismic=seis_match, reflectivity=r0_match)  # type: ignore
fig, axes = viz.plot_wavelet(pred_wlt, figsize=(6, 5), fmax=85, phi_max=100)
axes[2].set_ylim((-90, 5))

### Compute synthetic seismic 计算合成地震记录


In [ ]:
modeler = tutorial.get_modeling_tool()

synth_seismic = tie.compute_synthetic_seismic(modeler, pred_wlt, r0_match)
viz.plot_seismics(seis_match, synth_seismic, r0_match);  # type: ignore

### Similarity between real and synthetic seismic 真实地震记录与合成地震记录的相似性


In [ ]:
from wtie.optimize import similarity

xcorr = similarity.traces_normalized_xcorr(seis_match, synth_seismic)  # type: ignore
xcorr = grid.upsample_trace(xcorr, 0.001)
dxcorr = similarity.dynamic_normalized_xcorr(seis_match, synth_seismic)  # type: ignore
print("Max coeff of %.2f at a lag of %.3f sec" % (xcorr.R, xcorr.lag))  # type: ignore

### Visualize results 可视化结果


In [ ]:
fig, axes = viz.plot_tie_window(
    logset_twt,
    r0_match,
    synth_seismic,  # type: ignore
    seis_match,  # type: ignore
    xcorr,  # type: ignore
    dxcorr,
    figsize=(12, 6),
    wiggle_scale_syn=0.5,
    wiggle_scale_real=110000,
)
axes[0].locator_params(axis="y", nbins=16)
fig.tight_layout()